In [1]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=65b88bb3c3a0a4a026dc71ae97aaf9cd1ee3945ac7ffe94bdbaea576c267a903
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers



<h1>Input Training Data</h1>

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
df = pd.read_csv("train.csv")


 <h1>Input Model</h1>

In [3]:

from sentence_transformers import SentenceTransformer, SentencesDataset, LoggingHandler, losses



In [4]:
df_c = df[['Correct Statement']]
df_c['label'] = 1
df_i = df[['Incorrect Statement']].rename(columns={'Incorrect Statement':'Correct Statement'})
df_i['label'] = 2
pd.concat((df_c,df_i)).sample(frac=1.0)



<ipython-input-4-0f578d2335a5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_c['label'] = 1


,Correct Statement,label
1267,Some people has tiger as domesticated animals.,2
2799,He wiped his body with a towel,1
535,She washes her hair everyday with a mild shampoo,1
7815,My parents always praised me when I did well a...,1
6326,Bullfighter is bullfighting with red cloth,1
...,...,...
498,Many Americans can eat at toll booths.,2
6579,Family teachers teach children.,1
841,Men are called males,1
7744,We can always see stars when we look up at night,2


<h1>Finish Preparation</h1>

In [5]:


CS = []
for entry in df['Correct Statement']:
  CS.append([0, entry])

IS = []
for entry in df['Incorrect Statement']:
  IS.append([1, entry])

RR1 = []
for entry in df['Right Reason1']:
  CS.append([2, entry])

RR2 = []
for entry in df['Right Reason2']:
  CS.append([2, entry])

RR3 = []
for entry in df['Right Reason3']:
  CS.append([2, entry])

CR1 = []
for entry in df['Confusing Reason1']:
  CS.append([3, entry])

CR2 = []
for entry in df['Confusing Reason2']:
  CS.append([3, entry])


sentences = CS + IS 
# + RR1 + RR2 + RR3 + CR1 + CR2




<h1>Run Training Through Model</h1>

In [6]:
from sentence_transformers.readers.InputExample import InputExample
from torch.utils.data import DataLoader
all_examples = []
num = 0
for entry in sentences:
  example = InputExample(texts=[sentences[num][1]], label=sentences[num][0])
  all_examples.append(example)
  num += 1


  

train, test = train_test_split(all_examples, test_size = .3, shuffle = True)

print(train)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



<h1>Get Outputs</h1>

In [7]:
sentences = [[row[0],1-row[0],row[1]] for row in sentences]

In [8]:
[[[s[2]], s[0:2]] for s in sentences]

[[['He poured milk on his cereal.'], [0, 1]],
 [['He drinks milk.'], [0, 1]],
 [['Jeff ran a mile today'], [0, 1]],
 [['A mosquito stings me'], [0, 1]],
 [['A niece is a person.'], [0, 1]],
 [['A walk-in closet is larger than a normal closet.'], [0, 1]],
 [['I like to ride my bike'], [0, 1]],
 [['A GIRL WON THE RACE WITH HER FRIEND'], [0, 1]],
 [['he pour water in to the jug'], [0, 1]],
 [['A girl plays volleyball'], [0, 1]],
 [['Kids find eggs on Easter.'], [0, 1]],
 [['Mark drank water.'], [0, 1]],
 [['He went to the bank to withdraw cash.'], [0, 1]],
 [['An owner can price goods'], [0, 1]],
 [['she wear bangles in the hand'], [0, 1]],
 [['My cousin throws the snowball to my brother.'], [0, 1]],
 [['The caterpillar eats the rose bud'], [0, 1]],
 [['Lisa played jazz music for the President of the USA yesterday.'], [0, 1]],
 [['the computer broke down and the technician will fix it'], [0, 1]],
 [['I drank water.'], [0, 1]],
 [['I usually wear a shoes in my feet'], [0, 1]],
 [['My broth

In [9]:
np.array(sentences)[:,2]

array(['He poured milk on his cereal.', 'He drinks milk.',
       'Jeff ran a mile today', ...,
       'Harry went to the barbershop to have his glasses repaired',
       'Reilly is sleeping on the window', 'I have a desk on my lamp'],
      dtype='<U407')

In [10]:
from sentence_transformers import SentenceTransformer, SentencesDataset
from sentence_transformers import models, losses
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sentence_transformers.evaluation import BinaryClassificationEvaluator
import torch

model = SentenceTransformer(s)


# Define the loss function and data loader
#train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=2)
#train_examples = [InputExample(texts=[s[2]], label=s[0:2]) for s in sentences]
#train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)
#collate_fn=ConcatenatedEmbedding(model)

# Train the model
num_epochs = 5
#warmup_steps = int(len(50) * num_epochs * 0.1)  # 10% of train data for warm-up
embeddings= model.encode(np.array(sentences)[:,2])

# Evaluate the model on test data
#test_examples = [InputExample(texts=[sentences[num][1]], label=sentences[num][0]) for num in range(len(sentences))]
#test_dataset = SentencesDataset(test_examples, model)
#test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=32, collate_fn=ConcatenatedEmbedding(model))
##evaluator = BinaryClassificationEvaluator(test_dataloader)



In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X=embeddings
y =  np.array(sentences)[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

clf.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6425714285714286

<h1>Accuracy</h1>